# Fine tuning 

In [20]:
from datasets import load_dataset

dataset = load_dataset("mrm8488/CHISTES_spanish_jokes", split="train")
dataset

Dataset({
    features: ['id', 'text', 'keywords', 'funny', 'category'],
    num_rows: 2419
})

In [22]:
dataset.set_format('pandas')
df = dataset.to_pandas()
df.head(10)

,id,text,keywords,funny,category
0,0,"- ¡Rápido, necesitamos sangre!\n- Yo soy 0 pos...",sangre,1,otros
1,1,- ¿Cuál es el mejor portero del mundial? \n- E...,"futbol,porteros",1,otros
2,2,El otro día unas chicas llamarón a mi puerta y...,"dinero,agua",1,otros
3,3,"- Andresito, ¿qué planeta va después de Marte?...",planetas,1,profesiones
4,4,- ¿Por qué Bob Esponja no va al gimnasio? \n- ...,"esponja,gimnasios",1,otros
5,5,Van dos ciegos y le dice uno al otro: \n- Ojal...,ciegos,1,otros
6,6,Noticia de última hora!! \n\nMuere una suegra ...,"canarias,coches,noticias",2,familia
7,7,"– Mamá, mamá, en el colegio dicen que estoy lo...","locos,sillas",1,familia
8,8,"– Mamá, mamá, ¿me haces un bocata de jamón?\n–...","madres,jamón",1,otros
9,9,- Qué pasa si te expulsan de cuatro univerdade...,"universitarios,universidades",1,otros


## Preparando el conjunto de datos

In [23]:
import torch
import transformers
import keras
import accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch.nn as nn
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from typing import Optional, Tuple
import pandas as pd
import numpy as np

In [24]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "DeepESP/gpt2-spanish"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
model

/home/kevin/.pyenv/versions/3.9.15/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [6]:
def preprocess_function(max_len):
    def _preprocess_function(examples):
        return tokenizer(
            examples['text'],
            max_length=max_len,
            truncation=True,
            padding='longest',  # Usa el padding más corto posible para cada lote
            return_tensors='pt'
        )
    return _preprocess_function

In [27]:
dataset.reset_format()
tokenized_dataset = dataset.map(preprocess_function(max_len=256), batched=True)
tokenized_dataset = tokenized_dataset.remove_columns([col for col in tokenized_dataset.column_names if col != 'input_ids'])
tokenized_dataset = tokenized_dataset.train_test_split(train_size=0.70)
tokenized_dataset.set_format('torch')
tokenized_dataset

Map: 100%|██████████| 2419/2419 [00:00<00:00, 4317.48 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 1693
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 726
    })
})

In [28]:
# Muestra la primera entrada del dataset tokenizado
print(tokenized_dataset['train'][0])

# lo decodeamos
tokenizer.decode(tokenized_dataset['train'][0]['input_ids'])

{'input_ids': tensor([ 6907, 28370,   788,   289,   304,  1065,  8981,   268, 36175,  1925,
           21,   668,  5720,   297,  1742,   305,   725,   268,   370,  2148,
           35,   521,    23,   208,   703, 22266,  1408,   288,  1554,   268,
          304,  1079,   299,   788,   289,   297, 29522,   268,   304,  3050,
          324,   299,  7792,   307, 11900,   344,  6403,    23,   208, 34089,
          432,  1925,  6001,   281,  1593, 28370,   324,   426,  4013,    35,
          208,    22,   576, 13376,   299,  1481,   281, 11900,    40,   208,
           22,  4340,  2207,  1368,  2324,  1593, 28370,    23,   208,    22,
         7520,   913,  2953,   299,   366,  1368, 13365, 36175,  1925,    23,
          208,  6907, 28370, 18116,   304, 18597,   268,  4762, 47516,   596,
          288,  8460,   324,  2324,    35,   208,    22,   576,    42,   727,
        11979,    40,   208, 31361,   289,   297,  1095,   289,   299, 36175,
         1925,  6493,   325, 18597,   596,   288, 

'Rajoy está en un bar acompañado de Zapatero, cuando comienza el telediario de las 21:00.\nEl presentador cuenta la historia de un hombre que está en el ático de un edificio y que amenaza con saltar al vacío.\nZapatero mira a Rajoy y le pregunta:\n- ¿Crees que va a saltar?\n- Eso parece - dice Rajoy.\n- Pues yo creo que no - responde Zapatero.\nRajoy coloca un billete de 500 EUR sobre la barra y dice:\n- ¿Apostamos?\nJusto en el momento en que Zapatero pone su billete sobre la barra, el hombre salta y se mata.\nZapatero, muy afectado, le da su billete a Rajoy y le dice:\n- Una apuesta es una apuesta. Toma tu dinero.\nLuego, Rajoy admite:\n- Yo había visto el telediario de las 15:00 y sabía que se tiraría.\nZapatero responde:\n- Yo también lo había visto, pero jamás pensé que se volvería a tirar...<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftex

In [9]:
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

epochs = 15
batch_size = 8
logging_steps = len(tokenized_dataset['train']) // batch_size

# Definimos los parámetros globales de entrenamiento
training_args = TrainingArguments(
    output_dir='./hf-gpt',
    overwrite_output_dir=True,
    num_train_epochs=epochs,
    learning_rate=2e-5,
    per_device_eval_batch_size=batch_size,
    per_device_train_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    disable_tqdm=False,
    logging_steps=logging_steps
)

# Y definimos el entrenador, especificando el modelo, datasets y el tokenizador
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer
)

/home/kevin/.pyenv/versions/3.9.15/lib/python3.9/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [9]:
%%time
trainer.train()

 50%|████▉     | 272/546 [1:30:48<1:28:10, 19.31s/it]

{'loss': 3.4611, 'grad_norm': 4.214391708374023, 'learning_rate': 1.0036630036630037e-05, 'epoch': 1.0}


                                                     
 50%|█████     | 273/546 [1:33:45<1:05:19, 14.36s/it]

{'eval_loss': 3.2365965843200684, 'eval_runtime': 173.6238, 'eval_samples_per_second': 1.394, 'eval_steps_per_second': 0.179, 'epoch': 1.0}


100%|█████████▉| 544/546 [2:59:47<00:37, 18.99s/it]  

...

                                                   
100%|██████████| 546/546 [3:03:06<00:00, 14.13s/it]

{'eval_loss': 3.208950996398926, 'eval_runtime': 175.975, 'eval_samples_per_second': 1.375, 'eval_steps_per_second': 0.176, 'epoch': 2.0}


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
100%|██████████| 546/546 [3:03:08<00:00, 20.13s/it]

{'train_runtime': 10988.4916, 'train_samples_per_second': 0.396, 'train_steps_per_second': 0.05, 'train_loss': 3.309132829253927, 'epoch': 2.0}
CPU times: user 16h 17min 18s, sys: 1h 46min 28s, total: 18h 3min 47s
Wall time: 3h 3min 8s


TrainOutput(global_step=546, training_loss=3.309132829253927, metrics={'train_runtime': 10988.4916, 'train_samples_per_second': 0.396, 'train_steps_per_second': 0.05, 'total_flos': 1137665507328000.0, 'train_loss': 3.309132829253927, 'epoch': 2.0})

In [10]:
# Guardamos el modelo
model.save_pretrained('hf-gpt')

In [11]:
text = 'Habia una vez un hombre que'
best = 10

with torch.no_grad():
    tokens = tokenizer(text, return_tensors='pt')['input_ids'].to(device)
    print("Dimensiones de la entrada:", tokens.shape)
    output = model(input_ids=tokens)
    print("Dimensiones de la salida:", output.logits.shape)
    output = output.logits[0, -1, :]
    print("Dimensiones del último token de la secuencia:", output.shape)
    probs = torch.softmax(output, dim=-1)
    print("Dimensiones de la probabilidad de los tokens:", probs.shape)
    sorted_probs = torch.argsort(probs, dim=-1, descending=True)
    print({tokenizer.decode(token): f"{prob.cpu().numpy() * 100:.2f}%" for token, prob in zip(sorted_probs[:best], probs[sorted_probs[:best]])})

Dimensiones de la entrada: torch.Size([1, 7])
Dimensiones de la salida: torch.Size([1, 7, 50257])
Dimensiones del último token de la secuencia: torch.Size([50257])
Dimensiones de la probabilidad de los tokens: torch.Size([50257])
{' no': '10.73%', ' se': '9.15%', ' había': '3.97%', ' me': '2.61%', ' estaba': '2.59%', ',': '2.32%', ' era': '1.64%', ' tenía': '1.48%', ' ha': '1.42%', ' le': '1.24%'}


In [13]:
from typing import List, Tuple, Optional


def generate(
    model: nn.Module,  # El modelo de lenguaje neuronal utilizado para generar el texto
    # El tokenizador que convierte el texto en tokens para el modelo
    tokenizer: PreTrainedTokenizerBase,
    start: str,  # Texto de inicio que da el punto de partida para generar el chiste
    max_length: int = 50,  # Número máximo de tokens que se generarán
    eps: float = 0.5,  # Parámetro para la estrategia e-greedy de selección de tokens
    top_n: int = 5,  # Número de tokens con mayor probabilidad a considerar
    # Indica si se deben devolver los detalles de las iteraciones
    return_iterations: bool = False,
    # El dispositivo donde se ejecutará el modelo (por ejemplo, CPU o GPU)
    device: str = "cpu",
    # Lista de tokens que marcan el final del chiste
    stop_tokens: List[str] = ['.', '!', '?']
) -> Tuple[str, Optional[pd.DataFrame]]:

    # Inicializamos la lista 'output' con el texto inicial proporcionado
    output = [start]
    iterations = []  # Lista para almacenar los detalles de cada iteración si se solicita
    with torch.no_grad():  # Desactiva el cálculo de gradientes para ahorrar memoria, ya que no entrenamos el modelo
        # Convierte el último texto generado en tokens y los mueve al dispositivo especificado
        input_ids = tokenizer(
            output[-1], return_tensors='pt')['input_ids'].to(device)

        # Bucle principal para generar nuevos tokens hasta alcanzar la longitud máxima
        for _ in range(max_length):
            # Obtiene los logits (probabilidades sin normalizar) del modelo
            logits = model(input_ids=input_ids).logits
            # Aplica softmax para convertir los logits en probabilidades
            probs = torch.softmax(logits[0, -1, :], dim=-1)
            # Ordena los tokens por probabilidad, en orden descendente
            sorted_tokens = torch.argsort(probs, dim=-1, descending=True)

            # Estrategia e-greedy para seleccionar el siguiente token:
            # Si el valor aleatorio es menor que 'eps', se elige el token más probable,
            # de lo contrario, se selecciona uno basado en la distribución de probabilidades.
            if np.random.random_sample(1)[0] < eps:
                next_token = sorted_tokens[0].unsqueeze(
                    dim=0)  # Selecciona el token más probable
            else:
                # Selecciona un token basado en la probabilidad
                next_token = torch.multinomial(probs, 1)

            # Convierte el token seleccionado a texto
            next_word = tokenizer.decode(next_token)

            # Verifica si el token generado es un signo de puntuación que marca el final del chiste
            if any(stop in next_word for stop in stop_tokens):
                output.append(next_word)  # Añade el token final al output
                break  # Detiene la generación, ya que se alcanzó el remate del chiste

            # Si 'return_iterations' es True, almacenamos detalles de la iteración actual para análisis
            if return_iterations:
                # El texto generado hasta el momento
                iteration = {'input': ''.join(output)}
                # Obtiene los 'top_n' tokens más probables y sus respectivas probabilidades
                best_n = sorted_tokens[:top_n].cpu().tolist()
                # Guarda la información de cada token en el DataFrame
                choices = {f'Choice #{choice+1}': f'{tokenizer.decode(token)} ({prob:.4f})'
                           for choice, (token, prob) in enumerate(zip(best_n, probs[best_n].cpu().tolist()))}
                iteration.update(choices)
                iterations.append(iteration)

            # Añade la palabra generada al 'output'
            output.append(next_word)
            # Actualiza los 'input_ids' concatenando el nuevo token generado
            input_ids = torch.cat(
                [input_ids, next_token.unsqueeze(dim=0)], dim=-1)

        # Convierte la lista de tokens generados en una cadena de texto
        output_text = ''.join(output)

        # Devuelve el texto generado y, opcionalmente, las iteraciones si se solicitó
        if not return_iterations:
            return output_text, None
        else:
            # Convierte las iteraciones en un DataFrame
            df = pd.DataFrame(iterations)
            return output_text, df


# Ejemplo de uso
output_text, _ = generate(model, tokenizer, text,
                          max_length=50, eps=0.2, device=device)
print(output_text)

Habia una vez un hombre que no se había bañado ni una sola vez con él.
